# XGBoost avec labels encodés
Notebook pour convertir des labels non numériques en entiers (LabelEncoder), entraîner XGBoost, puis décoder les prédictions.

In [ ]:
# Imports et configuration
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from xgboost import XGBClassifier

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 40)


In [ ]:
# Chargement des données et inspection des labels

dataset_path = "../processed_data/csv_games_fusionnes.csv"
target_col = "rb1"  # à ajuster selon la cible à prédire

# Charger le dataset
df = pd.read_csv(dataset_path)
print(f"Dataset shape: {df.shape}")

# Vérifier la présence de la cible
if target_col not in df.columns:
    raise ValueError(f"La colonne cible '{target_col}' est absente du dataset")

# Afficher un aperçu des labels bruts
unique_labels = df[target_col].dropna().unique()
print(f"Nombre de labels uniques: {len(unique_labels)}")
print("Aperçu des labels:", unique_labels[:20])

In [ ]:
# Encodage numérique des classes pour XGBoost

# Filtrer les lignes avec label défini
mask = df[target_col].notna()
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df.loc[mask, target_col])

print(f"Labels encodés: {len(label_encoder.classes_)} classes")
print("Mapping (label -> id):")
for lbl, idx in zip(label_encoder.classes_[:10], range(min(10, len(label_encoder.classes_)))):
    print(f"  {lbl} -> {idx}")

In [ ]:
# Préparation des features et séparation train/test

feature_cols = [c for c in df.columns if c != target_col]
X_raw = df.loc[mask, feature_cols]

# Encodage one-hot pour les features catégorielles
X = pd.get_dummies(X_raw, drop_first=False)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"X train shape: {X_train.shape} | X test shape: {X_test.shape}")

In [ ]:
# Entraînement XGBoost avec labels encodés

num_classes = len(label_encoder.classes_)
model = XGBClassifier(
    objective="multi:softprob",
    num_class=num_classes,
    eval_metric="mlogloss",
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    tree_method="hist"
)

model.fit(X_train, y_train)
print("Modèle entraîné ✅")

In [ ]:
# Évaluation et décodage des prédictions

y_pred_ids = model.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_ids)
y_test_labels = label_encoder.inverse_transform(y_test)

acc = accuracy_score(y_test, y_pred_ids)
f1 = f1_score(y_test, y_pred_ids, average="macro")

print(f"Accuracy: {acc:.3f} | F1-macro: {f1:.3f}")
print("\nClassification report (labels décodés):")
print(classification_report(y_test_labels, y_pred_labels))